In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import json
import pandas as pd
import numpy as np

In [ ]:
with open("merged_data.json") as jfile:
    data = json.load(jfile)

In [ ]:
len(data)

In [ ]:
data[0]['tweet_data']["Images_in_tweet"]

In [ ]:
df = pd.DataFrame(index = range(13239), columns=["id", "original_id", "tweet_text", "tweet_url", "claim", "rating", "images", "text_images", "source"])

In [ ]:
def extract_labels(tweet):
    
    try:
        tweet_did = tweet["tweet_data"]["DtweetID"]
        tweet_oid = tweet["tweet_data"]["originalTweetID"]
        tweet_url = f"https://twitter.com/POTUS/status/{tweet_did}"
        tweet_text = tweet["tweet_data"]["OriginalTweetContent_Raw"]
        claim = tweet["article_data"]["claim"]
        rating = tweet["article_data"]["rating"]
        images = list(tweet["tweet_data"]["Images_in_tweet"].keys())
        text_images = [tweet["tweet_data"]["Images_in_tweet"][i]["text_for_the_image"] for i in images]
        source = tweet["article_data"]["url"]
        
        if len(images) == 1: images = images[0]
        if len(rating) == 1: rating = rating[0]
        if len(text_images) == 1: text_images = text_images[0] if len(text_images[0]) > 0 else None
            
        return [tweet_did, tweet_oid, tweet_text, tweet_url, claim, rating, images, text_images, source]     
    
    except Exception as e:
        return e


In [ ]:
data[0]["article_data"].keys()

In [ ]:
errors = {}
for i in df.index:
    info = extract_labels(data[i])
    
    if isinstance(info, list): 
        df.loc[i,] = np.asarray(info)
    else:
        errors[i] = info        



In [ ]:
errors

In [ ]:
df = df.drop(list(errors.keys()))

In [ ]:
df["label"] = [i[0].split("/")[-1].split(".")[0] for i in df.rating]
df["label"] = [i.replace("rating-","").replace("tom-","").replace("fom-","") for i in df.label]
df["label_info"] = [i[2] if len(i)>2 else None for i in df.rating]

In [ ]:
snopes = df[~df.source.str.contains("www.politifact.com")]

In [ ]:
snopes.label.value_counts()

In [ ]:
import json
with open("json_data.json") as rfile:
    data = json.load(rfile)
    

In [ ]:
data

In [ ]:
def map_availability(row):
    return [key for key in data.keys() if row in data[key]][0]

In [ ]:
snopes["available"] = snopes.id.apply(map_availability)

In [ ]:
# snopes.to_csv("snopes_clean.csv", index=False)

In [ ]:
# snopes = pd.read_csv("snopes_clean.csv")

In [ ]:
snopes

In [ ]:
snopes.available.value_counts()

In [ ]:
snopes.tail().source.tolist()

In [ ]:
snopes.tail().tweet_url.tolist()

In [4]:
!pip3 install requests tweepy==4.0 botometer

     |████████████████████████████████| 69 kB 2.0 MB/s eta 0:00:01
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
ERROR: botometer 1.6.1 has requirement tweepy<4,>=3.5.0, but you'll have tweepy 4.6.0 which is incompatible.


In [54]:
snopes = pd.read_csv("snopes_acc.csv")
snopes["botometer"] = ''

snopes_acc = snopes[snopes.acc_id.notna()]
snopes_acc["acc_id"] = snopes_acc.acc_id.astype(int)
accounts = snopes_acc.acc_id.tolist()


snopes_acc.reset_index(inplace=True)
del snopes_acc['index']

/var/folders/jf/w9ffcr5s1t1fffdv873rd0xr0000gn/T/ipykernel_17446/1629282148.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snopes_acc["acc_id"] = snopes_acc.acc_id.astype(int)


In [6]:
!pip install tqdm

  Using cached tqdm-4.63.0-py2.py3-none-any.whl (76 kB)


In [55]:
snopes_acc

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
0,1000037356836093953,1000023322481086466,Brave men died to keep us free. Radicals want ...,https://twitter.com/POTUS/status/1000037356836...,Liberals were offended by a World War I memori...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDL5a-XkAI7Hlw.jpg,#DontTearMeDown IRSTLIBERT . ORG DontTearMeDow...,https://www.snopes.com/fact-check/liberals-dem...,mixture,NaN,ok,3981225735,
1,1000098833798389760,1000059879682342912,“Stand for the National Anthem or be punished”...,https://twitter.com/POTUS/status/1000098833798...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDtKIRVAAA4kPm.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,1924071992,
2,1000158804795781120,1000073920760791040,@Humancipator35 @buddydusty @davidhogg111 🙌🏼 R...,https://twitter.com/POTUS/status/1000158804795...,"Emma Gonz lez, a survivor of the Parkland scho...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeD57XMV4AAUG4k.jpg,NaN,https://www.snopes.com/fact-check/emma-gonzale...,false,NaN,auth,924224208,
3,1000175568241872897,1000174645973315586,Just sayin' #NFL https://t.co/70tFG5ieMC,https://twitter.com/POTUS/status/1000175568241...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeFVdDlUwAA17qS.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,auth,170735298,
4,1000438777792344067,1000191784797040640,Those who do not learn from history are doomed...,https://twitter.com/POTUS/status/1000438777792...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeFkp5eU0AA2aLJ.jpg,"pnting n the wo "" ' t WHY to win . nbor mu t n...",https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,115563859,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6971,1000095067887910917,999866405687209984,History repeating itself with @realDonaldTrump...,https://twitter.com/POTUS/status/1000095067887...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeA9KrlVwAEHtPB.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,550658726,
6972,1000146882428096512,999903412425711616,I picked this up from someone here. Its🙄🤨🙄 wor...,https://twitter.com/POTUS/status/1000146882428...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeBe1X5VAAI-Zhh.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,91743765,
6973,1000018438184734720,999961946932723713,"Trump is a criminal, a gangster using criminal...",https://twitter.com/POTUS/status/1000018438184...,Notorious mobster Al Capone demanded an inquir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUF2ZV4AAMm0Z.jpg,Ci ) icago Sunday Cribune INAL 17 . Capone Dem...,https://www.snopes.com/fact-check/did-al-capon...,false,NaN,missing,881682896607600640,
6974,999992400725585920,999962442141782016,@VP @NFL @POTUS The only thing that’s #Winning...,https://twitter.com/POTUS/status/9999924007255...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUiW3VwAAQbyx.jpg,White Baseball Players Kneel in the 50 ' s to ...,https://www.snopes.com/fact-check/baseball-pla...,false,NaN,auth,115748564,


In [7]:
from tqdm import tqdm

In [4]:
pip install tweepy==4.0

  Using cached tweepy-4.0.0-py2.py3-none-any.whl (60 kB)
ERROR: botometer 1.6.1 has requirement tweepy<4,>=3.5.0, but you'll have tweepy 4.0.0 which is incompatible.
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install botometer

     |████████████████████████████████| 151 kB 7.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
df

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
0,1000037356836093953,1000023322481086466,Brave men died to keep us free. Radicals want ...,https://twitter.com/POTUS/status/1000037356836...,Liberals were offended by a World War I memori...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDL5a-XkAI7Hlw.jpg,#DontTearMeDown IRSTLIBERT . ORG DontTearMeDow...,https://www.snopes.com/fact-check/liberals-dem...,mixture,NaN,ok,3981225735,"{'cap': {'english': 0.7759887465100499, 'unive..."
1,1000098833798389760,1000059879682342912,“Stand for the National Anthem or be punished”...,https://twitter.com/POTUS/status/1000098833798...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDtKIRVAAA4kPm.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,1924071992,"{'cap': {'english': 0.7759887465100499, 'unive..."
2,1000158804795781120,1000073920760791040,@Humancipator35 @buddydusty @davidhogg111 🙌🏼 R...,https://twitter.com/POTUS/status/1000158804795...,"Emma Gonz lez, a survivor of the Parkland scho...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeD57XMV4AAUG4k.jpg,NaN,https://www.snopes.com/fact-check/emma-gonzale...,false,NaN,auth,924224208,"{'cap': {'english': 0.30618807248975083, 'univ..."
3,1000175568241872897,1000174645973315586,Just sayin' #NFL https://t.co/70tFG5ieMC,https://twitter.com/POTUS/status/1000175568241...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeFVdDlUwAA17qS.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,auth,170735298,"{'cap': {'english': 0.6763619064808265, 'unive..."
4,1000438777792344067,1000191784797040640,Those who do not learn from history are doomed...,https://twitter.com/POTUS/status/1000438777792...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeFkp5eU0AA2aLJ.jpg,"pnting n the wo "" ' t WHY to win . nbor mu t n...",https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,115563859,"{'cap': {'english': 0.7668769621923945, 'unive..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6971,1000095067887910917,999866405687209984,History repeating itself with @realDonaldTrump...,https://twitter.com/POTUS/status/1000095067887...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeA9KrlVwAEHtPB.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,550658726,NaN
6972,1000146882428096512,999903412425711616,I picked this up from someone here. Its🙄🤨🙄 wor...,https://twitter.com/POTUS/status/1000146882428...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeBe1X5VAAI-Zhh.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,91743765,NaN
6973,1000018438184734720,999961946932723713,"Trump is a criminal, a gangster using criminal...",https://twitter.com/POTUS/status/1000018438184...,Notorious mobster Al Capone demanded an inquir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUF2ZV4AAMm0Z.jpg,Ci ) icago Sunday Cribune INAL 17 . Capone Dem...,https://www.snopes.com/fact-check/did-al-capon...,false,NaN,missing,881682896607600640,NaN
6974,999992400725585920,999962442141782016,@VP @NFL @POTUS The only thing that’s #Winning...,https://twitter.com/POTUS/status/9999924007255...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachy

In [46]:
import botometer

rapidapi_key = "b0dd966c59mshdf2db2daf9a121fp15f57djsn5bf9d54db4aa"
consumer_key, consumer_secret = "uHktObbiyJpGoLUaENXdpUWyj", "DLoJ8VRhFUK4GQeo3XUP2LGaSN4D7a2mAZqSKlRQshr6N0Ru1r"
access_token, access_token_secret = "4121890515-8wV0UlTyu9hFzJL9d8NBU6BY24STUtS7zFeAcel", "vfnDrkEUAHtVayGJDZxau18mPjG4CuyB2JW6sG0xeYFGs"


twitter_app_auth = {
    'consumer_key': consumer_key, # 'sO59YfsV552iCsfLLpQTuS64T',
    'consumer_secret': consumer_secret, #'YVGEDbcA3Mg3bTY9rwXpLAH5ATOzbmhRBZ92wuWaBfVMdiflzq',
    'access_token': access_token, # 'AAAAAAAAAAAAAAAAAAAAAD%2FZWwEAAAAAJgG8hY2XJfmsYbbs%2Bp5BruRr8aM%3DtRxrtqs0eEhHQ9ZFE9U7tRlNnqm7oPUXeQEijSnj3RgxWgvN5l',
    'access_token_secret': access_token_secret,
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

# Check a single account by screen name
# result = bom.check_account('@clayadavis')

# Check a single account by id
# result = bom.check_account(1548959833)

# Check a sequence of accounts
# accounts = ['@clayadavis', '@onurvarol', '@jabawack']



# df = pd.read_csv("snopes_botometer_406.csv")

accounts = df.acc_id.tolist()

for i, account in tqdm(enumerate(accounts)):
    if i > 6837:
        print(i)
        try:
            result = bom.check_account(account)
            df.loc[i, "botometer"] = str(result)
        except Exception as e:
            print(f"Failed on account: {account}")
            print(e)
            df.loc[i, "botometer"] = str({'code': 34, 'message': 'Sorry, that page does not exist.'})
    
    
# for i, result in tqdm(enumerate(bom.check_accounts_in(accounts[407:]))):
#     print(accounts[(407+i)])
#     try:
# #         snopes_acc.loc[i, "botometer"] = str(result[1])
#         snopes_acc.loc[snopes_acc.acc_id.isin([result[0]]).index, "botometer"] = str(result[1])
#     except Exception as e:
#         print(e)
#         snopes_acc.loc[snopes_acc.acc_id.isin([result[0]]).index, "botometer"] = None


0it [00:00, ?it/s]

6838


6839it [00:03, 1813.70it/s]

6839
6840
Failed on account: 976183224228569216
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
6841
6842
6843


6844it [00:19, 264.71it/s] 

...

6974it [07:49,  3.94s/it]

6974


6975it [07:53,  3.89s/it]

6975


6976it [07:58, 14.58it/s]


In [47]:
i

6975

In [49]:
df.to_csv("snopes_final.csv", index=False)

In [48]:
df.tail()

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
6971,1000095067887910917,999866405687209984,History repeating itself with @realDonaldTrump...,https://twitter.com/POTUS/status/1000095067887...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeA9KrlVwAEHtPB.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,550658726,"{'cap': {'english': 0.3342309004634953, 'unive..."
6972,1000146882428096512,999903412425711616,I picked this up from someone here. Its🙄🤨🙄 wor...,https://twitter.com/POTUS/status/1000146882428...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeBe1X5VAAI-Zhh.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,91743765,"{'cap': {'english': 0.8054784364249085, 'unive..."
6973,1000018438184734720,999961946932723713,"Trump is a criminal, a gangster using criminal...",https://twitter.com/POTUS/status/1000018438184...,Notorious mobster Al Capone demanded an inquir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUF2ZV4AAMm0Z.jpg,Ci ) icago Sunday Cribune INAL 17 . Capone Dem...,https://www.snopes.com/fact-check/did-al-capon...,false,NaN,missing,881682896607600640,"{'cap': {'english': 0.4756769663222349, 'unive..."
6974,999992400725585920,999962442141782016,@VP @NFL @POTUS The only thing that’s #Winning...,https://twitter.com/POTUS/status/9999924007255...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUiW3VwAAQbyx.jpg,White Baseball Players Kneel in the 50 ' s to ...,https://www.snopes.com/fact-check/baseball-pla...,false,NaN,auth,115748564,"{'cap': {'english': 0.621687023966582, 'univer..."
6975,1000035228679852032,999973815655518213,@mjfree Sports has been used as a means of pro...,https://twitter.com/POTUS/status/1000035228679...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCejk4WkAMruuM.jpg,White Baseball Players Kneel in the 50 ' s to ...,https://www.snopes.com/fact-check/baseball-pla...,false,NaN,ok,518227176,"{'cap': {'english': 0.0, 'universal': 0.260367..."


In [5]:
df = pd.read_csv("snopes_4060.csv")

In [13]:
df.to_csv("snopes_4612.csv", index=False)

In [12]:
df.loc[4600:4620]

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
4600,917008562563543040,916890818048294912,@antipandemonium @ShaunKing @NFL @DonteStallwo...,https://twitter.com/POTUS/status/9170085625635...,NFL players did not stand for the national ant...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLlzVL0V4AEFNCG.jpg,NFL players only started coming on field for t...,https://www.snopes.com/fact-check/nfl-sideline...,mixture,NaN,ok,28722755,"{'cap': {'english': 0.5539725719744064, 'unive..."
4601,917134801416736768,917009839674544128,@HarryDowning113 @tia6sc @rodliz1225 @VP @POTU...,https://twitter.com/POTUS/status/9171348014167...,A meme accurately lists Democrats who have bee...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLnflb4UQAE6fY5.jpg,nwughtCrime Resistance In 1865 a Democrat kill...,https://www.snopes.com/fact-check/democrat-sho...,mostly-false,NaN,ok,422041473,"{'cap': {'english': 0.5778862084370772, 'unive..."
4602,919401783683936256,917081407960305665,Oh now you’ll stand? I think not. Hope you nev...,https://twitter.com/POTUS/status/9194017836839...,"In October 2017, Colin Kaepernick told a CBS r...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLogrJOV4AAZwvw.jpg,FOX NEWS Breaking News 2m ago Colin Kaepernick...,https://www.snopes.com/fact-check/kaepernick-a...,false,NaN,ok,166751745,"{'cap': {'english': 0.7920317685697057, 'unive..."
4603,917104093029699586,917102637950017536,@ProudResister You mean this guy? https://t.co...,https://twitter.com/POTUS/status/9171040930296...,A photograph shows a young Mike Pence with his...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLoz7n7UEAAOW6Y.jpg,Mike pence is trying to have this removed from...,https://www.snopes.com/fact-check/mike-pences-...,false,NaN,ok,867953768041660416,"{'cap': {'english': 0.22631829961109876, 'univ..."
4604,917189014918258688,917187370012823553,@ThePatriot143 Yeah &amp; this is the Army so ...,https://twitter.com/POTUS/status/9171890149182...,A photograph shows the Army football team knee...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLqBDJqWkAA0Oz0.jpg,Raymon King Jr 1 hr e U . S . Army Football Te...,https://www.snopes.com/fact-check/army-footbal...,false,NaN,auth,911585169060003840,"{'code': 34, 'message': 'Sorry, that page does..."
4605,917937955381088258,917202064358477824,@ChrisFagan57 @bridget4kicks @JulianAssange @S...,https://twitter.com/POTUS/status/9179379553810...,George Soros said that he funded the Black Liv...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLqOaQYV4AEzRea.jpg,""" I'm going to bring down the United States by...",https://www.snopes.com/fact-check/george-soros...,false,NaN,auth,54017596,"{'cap': {'english': 0.9158377984126878, 'unive..."
4606,917266413076623360,917248492900249600,@DanCorkill1 @TheGrungeHam @carol_NC66 As I ha...,https://twitter.com/POTUS/status/9172664130766...,The man accused of opening fire on hundreds of...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLq4c0CU8AAYLtT.jpg,NaN,https://www.snopes.com/fact-check/las-vegas-sh...,false,NaN,ok,124673927,"{'cap': {'english': 0.4197222421546159, 'unive..."
4607,917756421017690112,917248535136894977,it fucks me up that clickhole wrote one of the...,https://twitter.com/POTUS/status/9177564210176...,Barry Wilmore said that he saw astronaut Ramir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DLq4q82UMAAdf_0.jpg,"Barry Wilmore "" You never know true beauty unt...",https://www.snopes.com/fact-check/barry-wilmor...,false,NaN,auth,20600882,"{'cap': {'english': 0.7176911080698488, 'unive..."
4608,917388478144499717,917350015059791873,@USArmy333 @NBCNews @PeterAlexander @TODAYshow...,https://twitter.com/POTUS/status/9173884781444...,Prior to condemning flag 

In [6]:
pip install tweepy==3.6

  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.0.0
    Uninstalling tweepy-4.0.0:
      Successfully uninstalled tweepy-4.0.0
Note: you may need to restart the kernel to use updated packages.


In [164]:
df.botometer.isna().sum()

6232

In [167]:
df[df.acc_id.isin([3362345172])]

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
904,777593547142889472,777587316923600896,The @WSJ has different content in different lo...,https://twitter.com/POTUS/status/7775935471428...,The Wall Street Journal ran Donald Trump stori...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/CsqLirWUAAACzYg.jpg,TIIE WALL STREET JOIRNAL . THE LOAFER STEPS UP...,https://www.snopes.com/fact-check/wsj-differen...,true,NaN,ok,3362345172,"{'code': 34, 'message': 'Sorry, that page does..."


In [177]:
df.to_csv("snopes_1429.csv", index=False)

In [12]:
df.to_csv("snopes_1942.csv", index=False)

In [21]:
df.to_csv("snopes_2990.csv", index=False)

In [22]:
df = pd.read_csv("snopes_2990.csv")

In [23]:
df.iloc[2980:3000]

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
2980,851380310403108864,851369498351460352,"IF TOMAHAWK MISSLES ARE AROUND A $1M EACH, WHY...",https://twitter.com/POTUS/status/8513803104031...,CNN was caught using photographs of the same g...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9CsB6uXoAEaorL.jpg,NaN,https://www.snopes.com/fact-check/cnn-same-ref...,false,NaN,ok,113423108,"{'cap': {'english': 0.8076035454448582, 'unive..."
2981,851512935176425472,851501829351501824,United #UnitedAirlines https://t.co/GzXudvd9BD,https://twitter.com/POTUS/status/8515129351764...,United Airlines will begin charging for overhe...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9EkZW-WsAAXsa-.jpg,United Airlines just announced they will now b...,https://www.snopes.com/fact-check/united-airli...,mostly-false,NaN,ok,794706601676591104,"{'code': 34, 'message': 'Sorry, that page does..."
2982,851560765433806849,851555419457585153,"Honestly- I'm not a hateful person, but I hope...",https://twitter.com/POTUS/status/8515607654338...,"Whoopi Goldberg said that ""military widows lov...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9FVIyjVYAA6ZV6.jpg,".. JTHWITH JONES "" She was just looking for at...",https://www.snopes.com/fact-check/whoopi-goldb...,false,NaN,ok,59435428,"{'cap': {'english': 0.30618807248975083, 'univ..."
2983,851580394663182337,851567595409088518,@realDonaldTrump Were the crossed fingers behi...,https://twitter.com/POTUS/status/8515803946631...,"Neil Gorsuch called for the ""advancement of th...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9Ff_ZjXsAAp0EG.jpg,MakAusGr 8Again @user . Agaln Follow @user Ano...,https://www.snopes.com/fact-check/neil-gorsuch...,false,NaN,ok,3306164498,"{'cap': {'english': 0.4756769663222349, 'unive..."
2984,851574095145828352,851573350342242304,"Now how do you teach a squirrel anything, oh I...",https://twitter.com/POTUS/status/8515740951458...,A woman was arrested for training squirrels to...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9FlcdkVoAEzlwt.jpg,woMAN ARRESTED FOR TRAINING SQUIRRELS TO ATTAC...,https://www.snopes.com/fact-check/attack-squir...,false,NaN,missing,257715213,"{'cap': {'english': 0.8054784364249085, 'unive..."
2985,851729777727725568,851728536595345409,And the Australian Govt awards a $2b missile c...,https://twitter.com/POTUS/status/8517297777277...,President Trump owns stock in Raytheon and dir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9HykJAUMAA-LTi.jpg,"Meanwhile , the stocks of the military contrac...",https://www.snopes.com/fact-check/trump-raythe...,unproven,NaN,ok,285431964,"{'cap': {'english': 0.659646495284545, 'univer..."
2986,885580034399850496,851753867989721088,When CNN Uses The Same Girl In 3 Different Ref...,https://twitter.com/POTUS/status/8855800343998...,CNN was caught using photographs of the same g...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9IJlbHUIAA7Hzr.jpg,men CNN ugg game In 3 dffaent Refugee Crigig p...,https://www.snopes.com/fact-check/cnn-same-ref...,false,NaN,ok,785149463413268480,"{'cap': {'english': 0.7611878625273879, 'unive..."
2987,851818307522973696,851794590415724544,Annual budget of the National Endowment for th...,https://twitter.com/POTUS/status/8518183075229...,Providing security for First Lady Melania Trum...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/C9IuqehU0AA6VNz.jpg,NaN,https://www.snopes.com/fact-check/melania-trum...,unproven,NaN,ok,784890784273215488,"{'cap': {'english': 0.7546218700467833, 'unive..."
2988,852058180729413633,851914780914987010,#WhiteHelmets working w/victims of sarin gas ...,https://twitter.com/POTUS/status/8520581807294...,"The search-and-rescue organization in Syria, t..."

In [122]:
accounts[407]

9300262

In [120]:
snopes_acc.loc[406].acc_id

584177882

In [125]:
accounts[407:]

[9300262,
 721185346797572096,
 67899097,
 2499109494,
 2796929687,
 292929271,
 2269567158,
 2393098723,
 709920419529281536,
 4784596178,
 4236806414,
 494624290,
 561654065,
 1599141961,
 17860906,
 2839030037,
 4226960114,
 743688372460216320,
 118496889,
 2915156131,
 166329578,
 4676548814,
 4197546072,
 88494677,
 608471241,
 608471241,
 325164172,
 26672170,
 177584156,
 2801177472,
 95995039,
 126009130,
 126009130,
 399774230,
 972255098,
 1917731,
 65854394,
 741589503102570496,
 1917731,
 2244290719,
 701655880920510464,
 2793194082,
 18301196,
 1496756359,
 430118889,
 15164565,
 3697285157,
 2923563766,
 2923563766,
 1375629385,
 1325018372,
 25975551,
 22722452,
 374284455,
 49035775,
 48421473,
 428713002,
 3407847574,
 18545481,
 15164565,
 47652227,
 78523300,
 361275163,
 495622651,
 19098324,
 421275603,
 177584156,
 361275163,
 28905454,
 15164565,
 4429709805,
 1496756359,
 766797410,
 15164565,
 548172120,
 2241227238,
 15164565,
 30456403,
 248692627,
 28048355,

In [123]:
snopes_acc.loc[407]

id                                            758193276151222272
original_id                                   758190213973872640
tweet_text     Bill O'Reilly: Slaves who built White House we...
tweet_url      https://twitter.com/POTUS/status/7581932761512...
claim               The White House was built using slave labor.
rating         ['https://www.snopes.com/tachyon/2018/03/ratin...
images            http://pbs.twimg.com/media/CoWh-nhXEAA3jp1.jpg
text_images                                                  NaN
source         https://www.snopes.com/fact-check/was-the-whit...
label                                            mostly-true-new
label_info                                                   NaN
available                                                     ok
acc_id                                                   9300262
botometer                                                       
Name: 407, dtype: object

In [112]:
snopes_acc.to_csv("snopes_botometer_406.csv", index=False)

In [111]:
snopes_acc

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
0,1000037356836093953,1000023322481086466,Brave men died to keep us free. Radicals want ...,https://twitter.com/POTUS/status/1000037356836...,Liberals were offended by a World War I memori...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDL5a-XkAI7Hlw.jpg,#DontTearMeDown IRSTLIBERT . ORG DontTearMeDow...,https://www.snopes.com/fact-check/liberals-dem...,mixture,NaN,ok,3981225735,"{'cap': {'english': 0.7759887465100499, 'unive..."
1,1000098833798389760,1000059879682342912,“Stand for the National Anthem or be punished”...,https://twitter.com/POTUS/status/1000098833798...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDtKIRVAAA4kPm.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,1924071992,"{'cap': {'english': 0.7759887465100499, 'unive..."
2,1000158804795781120,1000073920760791040,@Humancipator35 @buddydusty @davidhogg111 🙌🏼 R...,https://twitter.com/POTUS/status/1000158804795...,"Emma Gonz lez, a survivor of the Parkland scho...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeD57XMV4AAUG4k.jpg,NaN,https://www.snopes.com/fact-check/emma-gonzale...,false,NaN,auth,924224208,"{'cap': {'english': 0.30618807248975083, 'univ..."
3,1000175568241872897,1000174645973315586,Just sayin' #NFL https://t.co/70tFG5ieMC,https://twitter.com/POTUS/status/1000175568241...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeFVdDlUwAA17qS.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,auth,170735298,"{'cap': {'english': 0.6763619064808265, 'unive..."
4,1000438777792344067,1000191784797040640,Those who do not learn from history are doomed...,https://twitter.com/POTUS/status/1000438777792...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeFkp5eU0AA2aLJ.jpg,"pnting n the wo "" ' t WHY to win . nbor mu t n...",https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,115563859,"{'cap': {'english': 0.7668769621923945, 'unive..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6971,1000095067887910917,999866405687209984,History repeating itself with @realDonaldTrump...,https://twitter.com/POTUS/status/1000095067887...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeA9KrlVwAEHtPB.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,550658726,
6972,1000146882428096512,999903412425711616,I picked this up from someone here. Its🙄🤨🙄 wor...,https://twitter.com/POTUS/status/1000146882428...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeBe1X5VAAI-Zhh.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,91743765,
6973,1000018438184734720,999961946932723713,"Trump is a criminal, a gangster using criminal...",https://twitter.com/POTUS/status/1000018438184...,Notorious mobster Al Capone demanded an inquir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUF2ZV4AAMm0Z.jpg,Ci ) icago Sunday Cribune INAL 17 . Capone Dem...,https://www.snopes.com/fact-check/did-al-capon...,false,NaN,missing,881682896607600640,
6974,999992400725585920,999962442141782016,@VP @NFL @POTUS The only thing that’s #Winning...,https://twitter.com/POTUS/status/9999924007255...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachyon/2018/0

In [110]:
test.loc[407].botometer

nan

In [ ]:
bots = snopes.botometer.tolist()

In [ ]:
dicts = []
for i in bots:
    if i:
        dicts.append(eval(i))
    else:
        dicts.append(None)


In [ ]:
for s in bom.check_account(accounts[0]):
    print(s)

In [ ]:
import botometer

rapidapi_key = "b0dd966c59mshdf2db2daf9a121fp15f57djsn5bf9d54db4aa"

consumer_key, consumer_secret = "uHktObbiyJpGoLUaENXdpUWyj", "DLoJ8VRhFUK4GQeo3XUP2LGaSN4D7a2mAZqSKlRQshr6N0Ru1r"
access_token, access_token_secret = "4121890515-8wV0UlTyu9hFzJL9d8NBU6BY24STUtS7zFeAcel", "vfnDrkEUAHtVayGJDZxau18mPjG4CuyB2JW6sG0xeYFGs"


twitter_app_auth = {
    'consumer_key': consumer_key, # 'sO59YfsV552iCsfLLpQTuS64T',
    'consumer_secret': consumer_secret, #'YVGEDbcA3Mg3bTY9rwXpLAH5ATOzbmhRBZ92wuWaBfVMdiflzq',
    'access_token': access_token, # 'AAAAAAAAAAAAAAAAAAAAAD%2FZWwEAAAAAJgG8hY2XJfmsYbbs%2Bp5BruRr8aM%3DtRxrtqs0eEhHQ9ZFE9U7tRlNnqm7oPUXeQEijSnj3RgxWgvN5l',
    'access_token_secret': access_token_secret,
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

accounts = snopes.acc_id.tolist()
for screen_name, result in tqdm(bom.check_accounts_in(accounts=accounts)):
    try:
        snopes.loc[i, "botometer"] = str(result[1])
        print(result)
    except Exception as e:
        print(e)
        snopes.loc[i, "botometer"] = None



In [50]:
rapidapi_key = "b0dd966c59mshdf2db2daf9a121fp15f57djsn5bf9d54db4aa"

consumer_key, consumer_secret = "uHktObbiyJpGoLUaENXdpUWyj", "DLoJ8VRhFUK4GQeo3XUP2LGaSN4D7a2mAZqSKlRQshr6N0Ru1r"
access_token, access_token_secret = "4121890515-8wV0UlTyu9hFzJL9d8NBU6BY24STUtS7zFeAcel", "vfnDrkEUAHtVayGJDZxau18mPjG4CuyB2JW6sG0xeYFGs"


twitter_app_auth = {
    'consumer_key': consumer_key, # 'sO59YfsV552iCsfLLpQTuS64T',
    'consumer_secret': consumer_secret, #'YVGEDbcA3Mg3bTY9rwXpLAH5ATOzbmhRBZ92wuWaBfVMdiflzq',
    'access_token': access_token, # 'AAAAAAAAAAAAAAAAAAAAAD%2FZWwEAAAAAJgG8hY2XJfmsYbbs%2Bp5BruRr8aM%3DtRxrtqs0eEhHQ9ZFE9U7tRlNnqm7oPUXeQEijSnj3RgxWgvN5l',
    'access_token_secret': access_token_secret,
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

result = bom.check_accounts_in(accounts[:3])



In [51]:
for a in result:
    print(a)

(3981225735, {'cap': {'english': 0.7759887465100499, 'universal': 0.8006881983123656}, 'display_scores': {'english': {'astroturf': 1.1, 'fake_follower': 1.4, 'financial': 0.0, 'other': 2.2, 'overall': 1.5, 'self_declared': 0.1, 'spammer': 0.1}, 'universal': {'astroturf': 0.8, 'fake_follower': 1.1, 'financial': 0.3, 'other': 3.0, 'overall': 1.8, 'self_declared': 0.1, 'spammer': 0.1}}, 'raw_scores': {'english': {'astroturf': 0.22, 'fake_follower': 0.29, 'financial': 0.01, 'other': 0.45, 'overall': 0.3, 'self_declared': 0.02, 'spammer': 0.02}, 'universal': {'astroturf': 0.15, 'fake_follower': 0.22, 'financial': 0.06, 'other': 0.6, 'overall': 0.37, 'self_declared': 0.02, 'spammer': 0.02}}, 'user': {'majority_lang': 'en', 'user_data': {'id_str': '3981225735', 'screen_name': 'TrumpStudents'}}})
(1924071992, {'cap': {'english': 0.7759887465100499, 'universal': 0.792436790569533}, 'display_scores': {'english': {'astroturf': 2.8, 'fake_follower': 0.4, 'financial': 0.6, 'other': 1.4, 'overall': 

In [52]:
a[1]

{'cap': {'english': 0.30618807248975083, 'universal': 0.37678996390568303},
 'display_scores': {'english': {'astroturf': 1.4,
   'fake_follower': 0.4,
   'financial': 0.3,
   'other': 0.9,
   'overall': 0.2,
   'self_declared': 0.0,
   'spammer': 0.0},
  'universal': {'astroturf': 1.1,
   'fake_follower': 0.6,
   'financial': 0.4,
   'other': 0.8,
   'overall': 0.3,
   'self_declared': 0.0,
   'spammer': 0.0}},
 'raw_scores': {'english': {'astroturf': 0.29,
   'fake_follower': 0.09,
   'financial': 0.06,
   'other': 0.18,
   'overall': 0.04,
   'self_declared': 0.0,
   'spammer': 0.0},
  'universal': {'astroturf': 0.22,
   'fake_follower': 0.12,
   'financial': 0.07,
   'other': 0.15,
   'overall': 0.06,
   'self_declared': 0.0,
   'spammer': 0.0}},
 'user': {'majority_lang': 'en',
  'user_data': {'id_str': '924224208', 'screen_name': 'jaybluuee'}}}

In [49]:
accounts

[3981225735,
 1924071992,
 924224208,
 170735298,
 115563859,
 839300788702875648,
 187286731,
 803984530700910592,
 432703985,
 96838049,
 22289130,
 1336826635,
 600061917,
 259132164,
 818231382,
 284236188,
 184383206,
 2316246829,
 807095,
 1266239359,
 3995778614,
 571135299,
 1367140224,
 1170769933,
 4041846647,
 2505411212,
 1102665366,
 22084427,
 227128602,
 221557378,
 704876221197193216,
 15317676,
 500391228,
 1260181622,
 22162379,
 14849146,
 2884176405,
 2411569569,
 2411569569,
 235709466,
 110855805,
 554987162,
 192529611,
 1329515617,
 554888488,
 1367140224,
 53159918,
 714647220268744704,
 1732751822,
 16973333,
 1893751014,
 4480863626,
 166329578,
 608471241,
 724755152515829760,
 128796694,
 607558430,
 124954456,
 4864871778,
 19874344,
 580312540,
 1909350781,
 17756195,
 304761097,
 350362817,
 4173349815,
 487600344,
 177584156,
 2682013394,
 4236806414,
 18545481,
 2255785398,
 4833119038,
 2416940856,
 457820209,
 30528628,
 3314758074,
 68450683,
 33906

In [ ]:
"@clayadavis"
[1548959833]

In [ ]:
pip install git+https://github.com/tweepy/tweepy.git


In [ ]:
accounts = snopes.original_id.tolist()



In [ ]:
import tweepy
from tqdm import tqdm

from time import sleep

consumer_key, consumer_secret = "uHktObbiyJpGoLUaENXdpUWyj", "DLoJ8VRhFUK4GQeo3XUP2LGaSN4D7a2mAZqSKlRQshr6N0Ru1r"
access_token, access_token_secret = "4121890515-8wV0UlTyu9hFzJL9d8NBU6BY24STUtS7zFeAcel", "vfnDrkEUAHtVayGJDZxau18mPjG4CuyB2JW6sG0xeYFGs"


bearer_token = "AAAAAAAAAAAAAAAAAAAAAD%2FZWwEAAAAA3Kiamz2egng9VKNdyiCM9%2BFUsrM%3DPS7mSlp9wPQ0BHX5inmPmOJBFrwevMrbEV4e9R10rEeU2DwaJt"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

client = tweepy.Client(bearer_token)
# client = tweepy.Client(auth)

# client = tweepy.Client(
#     consumer_key=consumer_key, consumer_secret=consumer_secret,
#     access_token=access_token, access_token_secret=access_token_secret
# )

# snopes["acc_id"] = None


for i, account in tqdm(enumerate(accounts)):
    
    if i<7250: continue
    
    if i % 250 == 0:
        sleep(900)
    try:
        response = client.get_tweet(id=account, expansions='author_id')
    except Exception as e:
        print("Cant get tweet -----------> ", e)
    
    if response.data:
        try:
            snopes.loc[i, "acc_id"] = response.includes["users"][0].id
        except Exception as e:
            snopes.loc[i, "acc_id"] = None
            print("Cant append to df :/ ----------> ", e)

In [10]:
# snopes.to_csv("snopes_acc.csv", index=False)

In [ ]:
first_100 = str(accounts[:100]).replace("[", "").replace("]", "").replace(" ", "")

In [31]:
accounts

[3981225735.0,
 1924071992.0,
 924224208.0,
 170735298.0,
 115563859.0,
 8.393007887028756e+17,
 187286731.0,
 8.039845307009106e+17,
 432703985.0,
 96838049.0,
 22289130.0,
 1336826635.0,
 600061917.0,
 259132164.0,
 818231382.0,
 284236188.0,
 184383206.0,
 2316246829.0,
 807095.0,
 1266239359.0,
 3995778614.0,
 571135299.0,
 1367140224.0,
 1170769933.0,
 4041846647.0,
 2505411212.0,
 1102665366.0,
 22084427.0,
 227128602.0,
 221557378.0,
 7.048762211971932e+17,
 15317676.0,
 500391228.0,
 1260181622.0,
 22162379.0,
 14849146.0,
 2884176405.0,
 2411569569.0,
 2411569569.0,
 235709466.0,
 110855805.0,
 554987162.0,
 192529611.0,
 1329515617.0,
 554888488.0,
 1367140224.0,
 53159918.0,
 7.146472202687447e+17,
 1732751822.0,
 16973333.0,
 1893751014.0,
 4480863626.0,
 166329578.0,
 608471241.0,
 7.247551525158298e+17,
 128796694.0,
 607558430.0,
 124954456.0,
 4864871778.0,
 19874344.0,
 580312540.0,
 1909350781.0,
 17756195.0,
 304761097.0,
 350362817.0,
 4173349815.0,
 487600344.0,
 1

In [14]:
list(set(snopes.botometer.tolist()))

['',
 "{'cap': {'english': 0.2520419704315795, 'universal': 0.5982419966968565}, 'display_scores': {'english': {'astroturf': 1.2, 'fake_follower': 0.2, 'financial': 0.0, 'other': 1.3, 'overall': 0.1, 'self_declared': 0.1, 'spammer': 0.0}, 'universal': {'astroturf': 0.6, 'fake_follower': 0.4, 'financial': 0.0, 'other': 0.9, 'overall': 0.7, 'self_declared': 0.1, 'spammer': 0.0}}, 'raw_scores': {'english': {'astroturf': 0.24, 'fake_follower': 0.05, 'financial': 0.0, 'other': 0.26, 'overall': 0.02, 'self_declared': 0.02, 'spammer': 0.0}, 'universal': {'astroturf': 0.12, 'fake_follower': 0.07, 'financial': 0.0, 'other': 0.18, 'overall': 0.14, 'self_declared': 0.02, 'spammer': 0.0}}, 'user': {'majority_lang': 'en', 'user_data': {'id_str': '14802766', 'screen_name': 'nan'}}}",
 "{'cap': {'english': 0.2787259978325127, 'universal': 0.6415870166679795}, 'display_scores': {'english': {'astroturf': 1.2, 'fake_follower': 0.4, 'financial': 0.0, 'other': 1.3, 'overall': 0.2, 'self_declared': 0.1, 's

In [15]:
snopes[snopes.botometer.str.contains("{'cap': {'english': 0.2787259978325127, 'universal': 0.6415870166679795}, 'display_scores': {'english': {'astroturf': 1.2, 'fake_follower': 0.4, 'financial': 0.0, 'other': 1.3, 'overall': 0.2, 'self_declared': 0.1, 'spammer': 0.0}, 'universal': {'astroturf': 0.6, 'fake_follower': 0.5, 'financial': 0.0, 'other': 1.0, 'overall': 0.8, 'self_declared': 0.1, 'spammer': 0.0}}, 'raw_scores': {'english': {'astroturf': 0.24, 'fake_follower': 0.08, 'financial': 0.0, 'other': 0.26, 'overall': 0.03, 'self_declared': 0.02, 'spammer': 0.0}, 'universal': {'astroturf': 0.12, 'fake_follower': 0.1, 'financial': 0.0, 'other': 0.19, 'overall': 0.16, 'self_declared': 0.02, 'spammer': 0.0}}, 'user': {'majority_lang': 'en', 'user_data': {'id_str': '14802766', 'screen_name': 'nan'}}}")]

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
199,740993500053901312,740992653555163136,@HillaryClinton Hillbilly Racists #blm #Black...,https://twitter.com/POTUS/status/7409935000539...,A photograph shows Hillary Clinton in blackfac...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/CkiI48wUkAIzNuC.jpg,THE DEMOCRATS AREN'T GOING TO LIKE THIS ONEI H...,https://www.snopes.com/fact-check/hillary-clin...,false,NaN,ok,NaN,"{'cap': {'english': 0.2787259978325127, 'unive..."
272,746201362900254720,746167181608685568,@GrantJr @wakeupfla @FoxNews here's the true R...,https://twitter.com/POTUS/status/7462013629002...,A photograph shows Hillary Clinton in blackfac...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/ClrrF4gWIAAgP-3.jpg,THE DEMOCRATS AREN'T GOING TO LIKE THIS ONE ! ...,https://www.snopes.com/fact-check/hillary-clin...,false,NaN,ok,NaN,"{'cap': {'english': 0.2787259978325127, 'unive..."
523,756589186979946496,756557780815867904,Guess This Applies To You Stupid &amp; Easy 2 ...,https://twitter.com/POTUS/status/7565891869799...,Hillary Clinton said that Democratic voters ar...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/Cn_VShHXgAA76R3.jpg,""" Look , the average Democrat voter is just pl...",https://www.snopes.com/fact-check/hillary-clin...,false,NaN,ok,NaN,"{'cap': {'english': 0.2787259978325127, 'unive..."
630,758755498300760064,758752442733457408,The Cosmic 'ice sculptures' of the Carina Nebu...,https://twitter.com/POTUS/status/7587554983007...,Scientists are stumped by a photograph taken b...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/CV5fNY5WcAQftf4.jpg,NaN,https://www.snopes.com/fact-check/gates-of-hea...,false,NaN,missing,NaN,"{'cap': {'english': 0.2787259978325127, 'unive..."
1061,770292202593484800,770282705434968064,@duceyco @garywhitta @SubjectPolitics PS: I'm ...,https://twitter.com/POTUS/status/7702922025934...,A photograph shows Hillary Clinton in blackfac...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/CrCXl2LUMAAkSWk.jpg,NaN,https://www.snopes.com/fact-check/hillary-clin...,false,NaN,ok,NaN,"{'cap': {'english': 0.2787259978325127, 'unive..."


In [23]:
snopes

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer
0,1000085962053959680,1000006390323253249,*cough* @NFL *cough* https://t.co/H9SskJ8q4Q,https://twitter.com/POTUS/status/1000085962053...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeC8Z5DX4AAduxj.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,auth,NaN,"{'cap': {'english': 0.2520419704315795, 'unive..."
1,1000057516615786496,1000015162714263553,@PhilMcCrackin44 Schiff's sister was married t...,https://twitter.com/POTUS/status/1000057516615...,Adam Schiff's sister Melissa was married to Ge...,['https://www.snopes.com/tachyon/2018/03/ratin...,['http://pbs.twimg.com/media/DeDEfF0XcAAHExJ.j...,"['', '', 'di "" twant wutted to cmt ur to frpyo...",https://www.snopes.com/fact-check/adam-schiffs...,false,NaN,auth,NaN,"{'cap': {'english': 0.2520419704315795, 'unive..."
2,1000037356836093953,1000023322481086466,Brave men died to keep us free. Radicals want ...,https://twitter.com/POTUS/status/1000037356836...,Liberals were offended by a World War I memori...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDL5a-XkAI7Hlw.jpg,#DontTearMeDown IRSTLIBERT . ORG DontTearMeDow...,https://www.snopes.com/fact-check/liberals-dem...,mixture,NaN,ok,3.981226e+09,"{'error': ""TweepError: [{'code': 34, 'message'..."
3,1000056983070892032,1000048498677571584,@AP @farmerfar55 Was he wearing the medal give...,https://twitter.com/POTUS/status/1000056983070...,A meme shows President Obama awarding the Pres...,['https://www.snopes.com/tachyon/2018/03/ratin...,['http://pbs.twimg.com/media/DeCrb0SUwAAsw-t.j...,"['', '', '', '']",https://www.snopes.com/fact-check/obama-medal-...,mostly-false,NaN,ok,NaN,"{'cap': {'english': 0.2520419704315795, 'unive..."
4,1000149982408794113,1000058582476435456,The importance of the electoral college is vit...,https://twitter.com/POTUS/status/1000149982408...,"Donald Trump won 3,084 of America's 3,141 coun...",['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeDr-FjVMAAlZe1.jpg,HILLARY'S POPULAR VOTE Mont . Ore . Idaho Wyo ...,https://www.snopes.com/fact-check/trump-clinto...,false,NaN,auth,NaN,"{'cap': {'english': 0.2520419704315795, 'unive..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11192,1000146882428096512,999903412425711616,I picked this up from someone here. Its🙄🤨🙄 wor...,https://twitter.com/POTUS/status/1000146882428...,A German football club was banned from playing...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeBe1X5VAAI-Zhh.jpg,FAILED TO GIVE NAZI SALUTE . German Football C...,https://www.snopes.com/fact-check/german-footb...,mostly-true-new,NaN,ok,9.174376e+07,
11193,1000018438184734720,999961946932723713,"Trump is a criminal, a gangster using criminal...",https://twitter.com/POTUS/status/1000018438184...,Notorious mobster Al Capone demanded an inquir...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUF2ZV4AAMm0Z.jpg,Ci ) icago Sunday Cribune INAL 17 . Capone Dem...,https://www.snopes.com/fact-check/did-al-capon...,false,NaN,missing,8.816829e+17,
11194,999992400725585920,999962442141782016,@VP @NFL @POTUS The only thing that’s #Winning...,https://twitter.com/POTUS/status/9999924007255...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachyon/2018/03/ratin...,http://pbs.twimg.com/media/DeCUiW3VwAAQbyx.jpg,White Baseball Players Kneel in the 50 ' s to ...,https://www.snopes.com/fact-check/baseball-pla...,false,NaN,auth,1.157486e+08,
11195,1000035228679852032,999973815655518213,@mjfree Sports has been used as a means of pro...,https://twitter.com/POTUS/status/1000035228679...,Baseball players in the 1950s were caught on c...,['https://www.snopes.com/tachyon/2018/03/ratin...,

In [19]:
snopes[snopes.original_id.isin([14802766])]

,id,original_id,tweet_text,tweet_url,claim,rating,images,text_images,source,label,label_info,available,acc_id,botometer


In [22]:
snopes.acc_id.tolist()[2]

3981225735.0

In [11]:
pip install tweepy==4.0

     |████████████████████████████████| 60 kB 1.5 MB/s eta 0:00:01
ERROR: botometer 1.6.1 has requirement tweepy<4,>=3.5.0, but you'll have tweepy 4.0.0 which is incompatible.
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.6.0
    Uninstalling tweepy-4.6.0:
      Successfully uninstalled tweepy-4.6.0
Note: you may need to restart the kernel to use updated packages.
